# 3.8 Almacenamiento y Mantenimiento

- Objetivo: estrategias de mantenimiento de índices, estadísticas y archivos de BD.
- Prerrequisitos: índices creados en tablas de hechos.
- Ejercicios: calcular fragmentación, rebuild vs reorganize (conceptual).
- Reto: calendario de mantenimiento preventivo.
- Errores comunes: rebuild innecesario de índices pequeños, no actualizar estadísticas, ventanas de mantenimiento inadecuadas.

## Fragmentación de índices (conceptual)

In [ ]:
-- Consulta de fragmentación (requiere permisos VIEW DATABASE STATE)
-- SELECT object_name(ips.object_id) AS tabla, i.name AS indice,
--        ips.avg_fragmentation_in_percent, ips.page_count
-- FROM sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, 'LIMITED') ips
-- JOIN sys.indexes i ON ips.object_id = i.object_id AND ips.index_id = i.index_id
-- WHERE ips.avg_fragmentation_in_percent > 10 AND ips.page_count > 100;

🟢 Ejercicio: Reglas de decisión para rebuild vs reorganize (fragmentación >30% rebuild; 10-30% reorganize).

🟠 Ejercicio: Actualización de estadísticas (UPDATE STATISTICS conceptual).

🔴 Reto: Diseñar calendario de mantenimiento semanal considerando ventanas de baja carga y prioridades (índices críticos primero).

## ¿Por qué mantener índices y estadísticas?

Con el tiempo, las bases de datos se **degradan** en rendimiento sin mantenimiento:

1. **Fragmentación de índices:** Las páginas de datos se desorganizan, causando más I/O (lectura de disco) para obtener mismos datos
2. **Estadísticas desactualizadas:** El optimizador de consultas toma decisiones basadas en distribución de datos que ya no refleja la realidad
3. **Crecimiento de logs:** Logs de transacciones sin respaldo pueden llenar el disco

**Para qué:**
- Mantener tiempos de respuesta consistentes (consulta que tardaba 2s no debe tardar 20s después de 6 meses)
- Evitar costos de I/O innecesarios (AWS RDS cobra por IOPS)
- Prevenir fallos por falta de espacio en disco
- Optimizar ventanas de backup y recuperación

**Cómo:**
- **Rebuild de índices:** Reconstruye índice completo, elimina fragmentación, compacta páginas
- **Reorganize de índices:** Reordena páginas existentes, menos intensivo que rebuild
- **Update Statistics:** Recalcula distribución de datos para optimizador de consultas
- **Shrink Files (con cuidado):** Reduce tamaño de archivos, puede causar fragmentación externa

## Fragmentación de Índices

### ¿Qué es fragmentación?

Cuando insertas, actualizas y eliminas registros, SQL Server puede:
- **Fragmentación lógica:** Páginas desordenadas (página 10 en disco físico contiene datos que deberían estar entre página 5 y 6)
- **Fragmentación de extensiones:** Extensiones (8 páginas) no contiguas en disco

**Impacto:**
- Lectura secuencial (range scan) requiere saltos aleatorios en disco (muy lento en discos mecánicos)
- Mayor uso de memoria (páginas fragmentadas ocupan más caché)

### Medir Fragmentación

```sql
SELECT 
  OBJECT_NAME(ips.object_id) AS tabla,
  i.name AS indice,
  ips.avg_fragmentation_in_percent AS fragmentacion_pct,
  ips.page_count AS num_paginas,
  ips.avg_page_space_used_in_percent AS espacio_usado_pct
FROM sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, 'LIMITED') ips
JOIN sys.indexes i ON ips.object_id = i.object_id AND ips.index_id = i.index_id
WHERE ips.index_id > 0  -- Excluir heaps (tablas sin clustered index)
ORDER BY ips.avg_fragmentation_in_percent DESC;
```

**Interpretación:**
- **< 10%:** Fragmentación insignificante, no requiere acción
- **10-30%:** Fragmentación moderada → REORGANIZE (online, menos intrusivo)
- **> 30%:** Fragmentación alta → REBUILD (offline en Standard Edition, online en Enterprise)

### Rebuild vs Reorganize

| Aspecto | REORGANIZE | REBUILD |
|---------|------------|---------|
| **Intensidad** | Baja (online siempre) | Alta (offline en Standard, online en Enterprise) |
| **Bloqueos** | Mínimos (permite lecturas/escrituras) | Shared/exclusive (bloquea escrituras en Standard) |
| **Fragmentación** | Reduce moderadamente | Elimina completamente |
| **Estadísticas** | No actualiza | Actualiza automáticamente (con FULLSCAN) |
| **Umbral** | 10-30% fragmentación | > 30% fragmentación |
| **Uso de disco** | No requiere espacio extra | Requiere ~1.5x tamaño del índice |

**Comando REORGANIZE:**
```sql
ALTER INDEX IX_fact_ventas_fecha ON dbo.fact_ventas REORGANIZE;
```

**Comando REBUILD:**
```sql
-- Standard Edition (offline)
ALTER INDEX IX_fact_ventas_fecha ON dbo.fact_ventas REBUILD;

-- Enterprise Edition (online, permite consultas durante rebuild)
ALTER INDEX IX_fact_ventas_fecha ON dbo.fact_ventas REBUILD WITH (ONLINE = ON);
```

**Rebuild todos los índices de una tabla:**
```sql
ALTER INDEX ALL ON dbo.fact_ventas REBUILD;
```

## Estadísticas

### ¿Qué son las estadísticas?

Histogramas que describen la distribución de valores en columnas indexadas. SQL Server los usa para estimar:
- Cuántas filas devolverá un filtro (`WHERE fecha = '2024-01-01'` → ~1000 filas)
- Qué índice usar (índice en fecha vs índice en cliente_id)
- Orden de JOINs (tabla pequeña → tabla grande)

### Problemas de estadísticas desactualizadas

Si insertaste 1 millón de registros pero estadísticas no se actualizaron:
- Optimizador cree que tabla tiene 100 registros
- Elige plan de ejecución para tabla pequeña (index seek)
- Plan real requiere table scan (lentísimo con 1 millón de registros)

### Actualizar estadísticas

```sql
-- Estadísticas de una tabla (sampleo automático)
UPDATE STATISTICS dbo.fact_ventas;

-- Estadísticas con FULLSCAN (más preciso, más lento)
UPDATE STATISTICS dbo.fact_ventas WITH FULLSCAN;

-- Estadísticas de todo el esquema
EXEC sp_updatestats;
```

**Frecuencia recomendada:**
- Tablas transaccionales (< 1M registros): AUTO_UPDATE_STATISTICS (activado por defecto)
- Tablas de hechos (> 10M registros): Manualmente después de cargas masivas
- Tablas particionadas: Actualizar por partición

## Estrategia de Mantenimiento

### Calendario Típico (Entorno 24/7)

**Diario (ventana de 2-4 AM, baja carga):**
- Backup de logs de transacciones (cada 1-2 horas en producción)
- Reorganize de índices con fragmentación 10-30% en tablas críticas (< 1 hora)

**Semanal (domingo 1-6 AM):**
- Rebuild de índices con fragmentación > 30%
- Update Statistics con FULLSCAN en tablas de hechos
- Verificación de integridad (DBCC CHECKDB)

**Mensual:**
- Análisis de crecimiento de archivos (proyectar necesidades de espacio)
- Revisión de índices no utilizados (sys.dm_db_index_usage_stats)
- Limpieza de índices duplicados o redundantes

### Script de Mantenimiento Automatizado (Conceptual)

```sql
-- Identificar índices que necesitan mantenimiento
DECLARE @sql NVARCHAR(MAX);

SELECT @sql = STRING_AGG(
  CASE 
    WHEN avg_fragmentation_in_percent > 30 AND page_count > 1000
    THEN 'ALTER INDEX ' + i.name + ' ON ' + OBJECT_SCHEMA_NAME(ips.object_id) + '.' + OBJECT_NAME(ips.object_id) + ' REBUILD;'
    WHEN avg_fragmentation_in_percent > 10 AND page_count > 1000
    THEN 'ALTER INDEX ' + i.name + ' ON ' + OBJECT_SCHEMA_NAME(ips.object_id) + '.' + OBJECT_NAME(ips.object_id) + ' REORGANIZE;'
  END, CHAR(13))
FROM sys.dm_db_index_physical_stats(DB_ID(), NULL, NULL, NULL, 'LIMITED') ips
JOIN sys.indexes i ON ips.object_id = i.object_id AND ips.index_id = i.index_id
WHERE ips.index_id > 0 AND page_count > 100;

-- Ejecutar comandos generados
EXEC sp_executesql @sql;
```

**Herramientas profesionales:**
- **SQL Server Maintenance Plans:** GUI para configurar tareas recurrentes
- **Ola Hallengren scripts:** Scripts open-source estándar de la industria (https://ola.hallengren.com)
- **Azure SQL Database:** Mantenimiento automático incluido

## Errores Comunes

1. **Rebuild innecesario de índices pequeños:**
   Índices con < 100 páginas no se benefician de rebuild (overhead mayor que beneficio).

2. **No verificar espacio disponible antes de REBUILD:**
   REBUILD requiere ~1.5x el tamaño del índice en disco. Puede fallar por falta de espacio.

3. **Ejecutar mantenimiento en horario pico:**
   REBUILD offline bloquea tabla. Programar en ventanas de baja carga.

4. **No actualizar estadísticas después de cargas masivas:**
   Insertar 10M registros sin UPDATE STATISTICS puede causar planes de ejecución pésimos.

5. **Usar SHRINK DATABASE regularmente:**
   Shrink causa fragmentación externa. Usar solo después de eliminar grandes volúmenes de datos (no como mantenimiento rutinario).

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](07_query_tuning.ipynb) | [Siguiente ➡️](09_observabilidad_monitoreo.ipynb)
---
